## load libraries Pythons in Jupyter

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import glob
import os 
import kaleido
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import Image
tqdm.pandas()

In [2]:
from bokeh.sampledata.autompg import autompg_clean as dfac
import hvplot.pandas
import panel as pn
import holoviews as hv
hv.extension('bokeh')
pn.extension('tabulator',"plotly")
PALETTE = ['#FF6F69', '#FFCC5C', '#88D8B0']

### Define function to detrminate properties

In [3]:
def environment():
    try:
        get_ipython()
        return "notebook"
    except:
        return "server"
environment()

'notebook'

## Load tables and other datas 

In [4]:
modelos = ['RF', 'GTB']
vers = ['5']
nameBacias = [
      '741', '7421','7422','744','745','746','751','752',  # '7492',
      '753', '754','755','756','757','758','759','7621','7622','763',
      '764','765','766','767','771','772','773', '7741','7742','775',
      '776','76111','76116','7612','7613','7614','7615',  # '777','778',
      '7616','7617','7618','7619'
]
nameBacias = [int(kk) for kk in nameBacias]
lstYear =  [kk for kk in range(1985, 2023)]

In [5]:
def getPathCSV (nfolders):
    # get dir path of script 
    mpath = os.getcwd()
    # get dir folder before to path scripts 
    pathparent = str(Path(mpath).parents[0])
    # folder of CSVs ROIs
    roisPathAcc = pathparent + '/dados/' + nfolders
    return pathparent, roisPathAcc
# help(os)
base_path, input_path_CSVs = getPathCSV('acc/ptosAccCol9/')
print("path the base ", base_path)
print("path of CSVs from folder ", input_path_CSVs)

path the base  /home/superuser/Dados/mapbiomas/dev_colection9_mapbiomas/src
path of CSVs from folder  /home/superuser/Dados/mapbiomas/dev_colection9_mapbiomas/src/dados/acc/ptosAccCol9/


In [7]:
pathOutpout = base_path + '/dados/accTables/'
nameTablesGlob = "regMetricsAccGlobalCol9.csv"        
nameTablesBacia = "regMetricsAccBaciasCol9.csv"

In [12]:
dfAccYY = pd.read_csv(pathOutpout + nameTablesGlob)
dfAccBacia = pd.read_csv(pathOutpout + nameTablesBacia)
# show the first rows     
print("registros Agrupados \n")
colInts = [kk for kk in dfAccBacia.columns]
print("colunas listadas \n   ==> ",colInts)
colInts.remove('Unnamed: 0')
dfAccBacia = dfAccBacia[colInts]
dfAccBacia.head()


registros Agrupados 

colunas listadas 
   ==>  ['Unnamed: 0', 'version', 'Models', 'Bacia', 'Years', 'Accuracy', 'Accuracy_Bal', 'Precision', 'ReCall', 'F1-Score', 'Jaccard']


,version,Models,Bacia,Years,Accuracy,Accuracy_Bal,Precision,ReCall,F1-Score,Jaccard
0,5,GTB,741,1985,0.580645,0.627174,0.479167,0.418116,0.436325,0.357222
1,5,GTB,741,1986,0.580645,0.473485,0.341667,0.315657,0.326984,0.287749
2,5,GTB,741,1987,0.612903,0.648810,0.522222,0.432540,0.443915,0.371934
3,5,GTB,741,1988,0.806452,0.744048,0.534127,0.496032,0.511905,0.443237
4,5,GTB,741,1989,0.870968,0.912500,0.689474,0.730000,0.689377,0.621429


In [14]:
dfAccYY.head()

,version,Models,Bacia,Years,Accuracy,Accuracy_Bal,Precision,ReCall,F1-Score,Jaccard
0,5,GTB,Caatinga,1985,0.641782,0.470114,0.375963,0.470114,0.394581,0.296540
1,5,GTB,Caatinga,1986,0.637946,0.450614,0.368860,0.450614,0.385435,0.291551
2,5,GTB,Caatinga,1987,0.659192,0.469574,0.394068,0.469574,0.404199,0.307327
3,5,GTB,Caatinga,1988,0.662437,0.462915,0.396828,0.462915,0.405876,0.311633
4,5,GTB,Caatinga,1989,0.649749,0.470309,0.388113,0.470309,0.400150,0.304554


In [13]:
dfAccYY['Bacia'] = ['Caatinga'] * dfAccYY.shape[0]
dfAccYY = dfAccYY[colInts]

In [17]:
dfAcc = pd.concat([dfAccYY, dfAccBacia], ignore_index=True, axis=0)
dfAcc.head(2)

,version,Models,Bacia,Years,Accuracy,Accuracy_Bal,Precision,ReCall,F1-Score,Jaccard
0,5,GTB,Caatinga,1985,0.641782,0.470114,0.375963,0.470114,0.394581,0.296540
1,5,GTB,Caatinga,1986,0.637946,0.450614,0.368860,0.450614,0.385435,0.291551


In [9]:
dfAcc['Models'].unique()

array(['GTB', 'RF'], dtype=object)

In [18]:
dfAccBacia[dfAccBacia['Bacia']== 76116].shape

(76, 10)

https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix </br>
https://pieriantraining.com/confusion-matrix-with-scikit-learn-and-python/


<table style="text-align:center">
    <tr>
        <th> </th>
        <th><h3>Predicted Positive</h3></th>
        <th><h3>Predicted Negative</h3> </th>
    </tr>
     <tr>
        <th>Actual Positive </th>
        <th>True Positive (TP) </th>
        <th>False Negative (FN) </th>
    </tr>
     <tr>
        <th>Actual Negative </th>
        <th>False Positive (FP) </th>
        <th>True Negative (TN) </th>
    </tr>
</table>


<h2 style="text-align:center"> Accuracy </h2>
$$
Accuracy = \frac{(Number\ of\ Correctly\ Classified\ Samples)}{(Total\ Number\ of\ Samples)}
$$
<h2 style="text-align:center"> Precision</h2>
$$
Precision =  \frac{(True\ Positives)}{(True\ Positives + False\ Positives)}
$$
<h2 style="text-align:center"> Recall</h2>
$$
Recall =  \frac{(True\ Positives)}{(True\ Positives + False\ Negatives)}
$$
<h2 style="text-align:center"> F1-Score</h2>
$$
F1-score =  \frac{2 * ((Precision * Recall)}{(Precision + Recall))}
$$

In [31]:
def get_chart_Plot_plotlyX(nbacia= 741, nModel= "RF"):
    colunas = ["Accuracy","Accuracy_Bal","Precision","ReCall", "F1-Score","Jaccard"]
    dfTemp = dfAcc[(dfAcc["Bacia"] == nbacia) & (dfAcc["Models"] == nModel)]
    # print(dfTemp.head())
    fig = px.line(dfTemp, 
                x="Years", y=colunas,
                hover_data={"Years": "%Y"},
                title=  f'Accuracy metrics of basin {nbacia} with model {nModel} '.upper(),
                template="plotly_dark",
            )
    fig.update_xaxes(
        dtick="M1",
        tickformat="%b\n%Y"
    )
    # fig.show()
    fig.layout.autosize = True
    return fig

In [32]:
# dftGG  = filteringDataF(741, "RF")
# print(dftGG.head())
# get_chart_Plot_plotlyX(dftGG)

In [33]:
# chart = pn.pane.Vega(
#     pn.bind(filteringDataF, wgBacia, wgModels), min_height=400, max_height=800, sizing_mode='stretch_width'
# )

In [37]:
from bokeh.models import MultiSelect, Tabs

In [34]:
lstBaciasUpdate = ['Caatinga'] + nameBacias

In [35]:
wgBacia = pn.widgets.Select(name='name_bacias', options=lstBaciasUpdate, value=nameBacias[0], width=200)
# wgModels = pn.widgets.Select(name='modelo', options=modelos, value='GTB')
wgModels = pn.widgets.RadioButtonGroup(
    description="Classifications Models",
    name="modelo", options=modelos, width=200).servable()
intro = pn.pane.Markdown("""
This app provides the interactive plot of **Accuracy from Collections Maps 9** for  .\n\n
""", sizing_mode='stretch_width')

In [38]:
# slider = pn.widgets.IntSlider(value=5, start=1, end=5)
interactive_model = pn.bind(get_chart_Plot_plotlyX, wgBacia, wgModels)
layoutAcc = pn.Row( 
            pn.Column(intro, wgModels, wgBacia).servable(),
            pn.Column(pn.pane.Plotly(interactive_model), width=850, height=450 )
)



In [39]:
tabs = Tabs(tabs=[("Accuracy", layoutAcc)], width=850, height=450)
tabs

ValueError: failed to validate Tabs(id='00d64180-1618-43c5-991c-86fc4c34001d', ...).tabs: expected an element of List(Instance(TabPanel)), got seq with invalid items [('Accuracy', Row
    [0] Column
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] RadioButtonGroup(description='Classifications Models', name='modelo', options=['RF', 'GTB'], value='RF', width=200)
        [2] Select(name='name_bacias', options=['Caatinga', 741, ...], value=756, width=200)
    [1] Column(height=450, sizing_mode='fixed', width=850)
        [0] Plotly(Figure))]